# Метод внешних штрафов

In [2]:
import numpy as np
import numpy.linalg as lg

## Функция и ее градиент

In [3]:
def fx(x):
    return x[0] ** 2 + x[1] ** 2 - 10 * x[0] - 8 * x[1]

def gradfx(x):
    arr = np.array([2 * x[0] - 10, 
                    2 * x[1] - 8, 
                    0 
                   ])
    return arr;

## Условие и его градиент

In [4]:
def Hx(x):
    return (max(0, -x[0])) ** 2 + (max(0, -x[1])) ** 2 + (max(0, -x[2])) ** 2 + (2 * x[0] + 3 * x[1] + x[2] - 6) ** 2

def gradHx(x):
    arr = np.array([- 2 * max(0, -x[0]) + 4 * (2 * x[0] + 3 * x[1] + x[2] - 6),
                    - 2 * max(0, -x[1]) + 6 * (2 * x[0] + 3 * x[1] + x[2] - 6),
                    - 2 * max(0, -x[2]) + 2 * (2 * x[0] + 3 * x[1] + x[2] - 6)
                    ])
    return arr;

## вспомогательные функции

In [5]:
def mygeom(x0, e, grad):
    ##print('    start mygeom')
    a = 0
    scalar = abs(sum(grad(x0)*grad(x0 - a * grad(x0))))
    while scalar > e: 
        a+=e
        scalar1 = scalar
        scalar = abs(sum(grad(x0)*grad(x0 - a * grad(x0))))
        #print('    a = ', a)#print('    scalar = ', scalar)
        if scalar1 < scalar:
            #print('    break', scalar)#print('        break', scalar) #print('        break', scalar1)
            break
    ##print('    a = ', a)
    ##print('    scalar = ', scalar)
    ##print('    end mygeom')
    return a
#------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------#
def func(f, x0, gradx0):
    def fun(a):
        A = x0 -  gradx0 * a
        return f(A)
    return fun
#------------------------------------------------------------------------------------------------------------#
# первая производная
#------------------------------------------------------------------------------------------------------------#
def der1_e(f_, x0, a, gradx0, e):
    e = e / 2
    f_(x0)
    f = func(f_, x0, gradx0)
    def der(a):
        return (f(a + e) - f(a - e)) / (e * 2)
    return der
#------------------------------------------------------------------------------------------------------------#
# вторая производная
#------------------------------------------------------------------------------------------------------------#
def der2_e(f_, x0, a, gradx0, e):
    e = e / 2
    df = der1_e(f_, x0, a, gradx0, e)
    def der2(a):
        return (df(a + e) - df(a - e)) / ( e * 2 )
    return der2
#------------------------------------------------------------------------------------------------------------#
# ищем а при одномерной минимизации по направлению
#------------------------------------------------------------------------------------------------------------#
def grad_descent_a1(f, x0, x1, e):
    a = 0
    df = der1_e(f, x0, a, x1-x0, e / 5)
    while e < abs(df(a)):
        d2f = der2_e(f, x0, a, x1-x0, e / 5)
        a = a - df(a)/d2f(a)
        df = der1_e(f, x0, a, x1-x0, e / 5)
    return a
#------------------------------------------------------------------------------------------------------------#
# тоже самое, только на вход подается начало вектора и сам вектор, а не его начало и конец
#------------------------------------------------------------------------------------------------------------#
def grad_descent_a2(f, x0, vec, e):
    a = 0
    df = der1_e(f, x0, a, vec, e / 5)
    while e < abs(df(a)):
        d2f = der2_e(f, x0, a, vec, e / 5)
        a = a - df(a)/d2f(a)
        df = der1_e(f, x0, a, vec, e / 5)
    return a

## МНГС

In [6]:
def min_gradient(f, grad, x0, e):
    n = 0
    while abs(lg.norm(grad(x0))) > e:
        n+=1
        print('')
        print('------[----', n, '----]------')
        #a = mygeom(x0, e/1000, grad)
        a = grad_descent_a2(f, x0, grad(x0), e/100)
        print('')
        print('a = ', a)
        print('  x  = ', x0)
        print('f(x) = ', f(x0))
        print(' |grad(x)| = ', lg.norm(grad(x0)))
        x0 = x0 - a * grad(x0)
    print('сделано ', n, ' шагов')
    return x0

### Ускоренный градиентный метод p-того порядка

In [37]:
#------------------------------------------------------------------------------------------------------------#
def p_gradient(f, grad, x0, e):
    n  = 0
    n1 = 0;
    while abs(lg.norm(grad(x0))) > e:
        x1 = x0
        
        for k in range(len(x0) * 4):
            if(abs(lg.norm(grad(x1))) <= e):
                x0 = x1
                break
            n+=1
            print('')
            print('------[----', n, '----]------')
            #a = mygeom(x1, e/1000, grad)
            a = grad_descent_a2(f, x1, grad(x1), e / 1000)
            x1 = x1 - grad(x1) * a

            print('')
            print('a = ', a)
            print('  x  = ', x1)
            print('f(x) = ', f(x1))
            if f(x1) >= f(x0):
                break
        if(abs(lg.norm(grad(x0))) <= e):
            break
        if f(x1) >= f(x0):
            break
        print('-------------------------------------------------------------------')
        print('----------------------------скачок[----', n1, '----]скачок')
        a = grad_descent_a1(f, x0, x1, e / 100)
        x0 = x0 -  (x1 - x0) * a
        print('----------------------------  a1  = ', a)
        print('----------------------------  x1  = ', x0)
        print('----------------------------f(x1) = ', f(x0))
        n1+=1
        
    print('сделано ', n, ' шагов')
    print('сделано ', n1, ' скачков')
    return x0
#------------------------------------------------------------------------------------------------------------#



#------------------------------------------------------------------------------------------------------------#
def ravine_gradient(f, grad, x0, e):
    x1 = x0 - e*10
    n  = 0
    n1 = 0
    while abs(lg.norm(grad(x0))) > e:
        for k in range(len(x0)):
            if(abs(lg.norm(grad(x0))) < e):
                break
            n+=1
            print('')
            print('------[----', n, '----]------')
            
            #a = mygeom(x0, e/100, grad)
            a = grad_descent_a2(f, x1, grad(x1), e / 1000)
            x0 = x0 - grad(x0) * a
            
            #a = mygeom(x1, e/100, grad)
            a1 = grad_descent_a2(f, x1, grad(x1), e / 1000)
            x1 = x1 - grad(x1) * a1
            
            print('  x0    = ', x0 ,   ' x1    = ', x1 )
            print('  f(x0) = ', f(x0), ' f(x1) = ', f(x1) )
            
        if(abs(lg.norm(grad(x0))) < e):
                break
                
        print('')
        print('скачок[----', n1, '----]скачок')
        a = grad_descent_a1(f, x0, x1, e / 100)
        x0 = x0 - a * (x1-x0)
        x1 = x0 - e*10
        print('a1 = ', a)
        print('  x1  = ', x0)
        print('f(x1) = ', f(x0))
        n1+=1
        
    print('сделано ', n, ' шагов')
    print('сделано ', n1, ' скачков')
    return x0
#------------------------------------------------------------------------------------------------------------#

## Реализаия метода

### Суммарная функция *f(x)+rH(x)*

In [38]:
def sumf(f, r , H):
    def result(x):
        res = f(x) + r * H(x)
        return res
    return result

In [95]:
def r_0(n):
    return np.sqrt(np.sqrt(n))
def r_1(n):
    return np.sqrt(n)
def r_2(n):
    return ((n * 2 + 10) ** (1/2) + 10)
def r_3(n):
    return ((n * 2 + 5 ) ** (1/2) + 5 )
def r_active(n):
    return ((n * 2 + 6 ) ** (1/2) + 6 )
def r_5(n):
    return ((n * 2 + 10) ** (1/2) + 10)

def sanction( f_, gradf_, H, gradH, x, e):
    n = 1
    r = r_active(n)
    f = sumf(f_, r, H)
    grad = sumf(gradf_, r, gradH)
    
    while( H(x) > e) or (lg.norm(grad(x)) > e):
        print('------------------------------------------------------------------------[----', n, '----]------')
        #x = min_gradient(f, grad, x, e)#лучше разделить на 10
        #x = p_gradient(f, grad, x, e)
        #x = ravine_gradient(f, grad, x, e)
        n += 1
        r = r_active(n)
        f = sumf(f_, r, H)
        grad = sumf(gradf_, r, gradH)
        
        print(' -------------------------------------------------------------- f(x) + (( ', r ,' )) * H(x)    =    ', f(x))
        print(' --------------------------------------------------------------   x       = ', x)
        print(' -------------------------------------------------------------- H(x)      = ', H(x))
        print(' -------------------------------------------------------------- |grad(x)| = ', lg.norm(grad(x)))
        
          
    print('алгоритм был выполнен ( ', n - 1, ' ) раз')
    return x

## тесты

In [96]:
x0 = np.array([33/13, 4/13, 0.])
print('x0         = ', x0)
print('fx(x0)     = ', fx(x0))
print('H(x0)      = ', Hx(x0))
print('gradfx(x0) = ', gradfx(x0))
print('gradHx(x0) = ', gradHx(x0))

x0         =  [2.53846154 0.30769231 0.        ]
fx(x0)     =  -21.307692307692307
H(x0)      =  0.0
gradfx(x0) =  [-4.92307692 -7.38461538  0.        ]
gradHx(x0) =  [0. 0. 0.]


In [97]:
x0 = np.array([3., 3.7, 2.])
e = 0.05
x_min = sanction(fx, gradfx, Hx, gradHx, x0, e)
print(round(fx(x_min), 3), x_min)

------------------------------------------------------------------------[---- 1 ----]------

------[---- 1 ----]------

a =  0.004015272791243125
  x  =  [3.  3.7 2. ]
f(x) =  1478.136378877694
 |grad(x)| =  862.8496234621132

------[---- 2 ----]------

a =  0.49988912729649465
  x  =  [1.15855743 0.91615367 1.07124817]
f(x) =  -16.56793541683677
 |grad(x)| =  3.8899456112449244

------[---- 3 ----]------

a =  0.003994479982709739
  x  =  [ 2.58380213  0.3762971  -0.13642486]
f(x) =  -21.64024522755786
 |grad(x)| =  1.5375648548386744
сделано  3  шагов
 -------------------------------------------------------------- f(x) + ((  9.16227766016838  )) * H(x)    =     -21.632326206891705
 --------------------------------------------------------------   x       =  [ 2.58052548  0.37137738 -0.1380926 ]
 -------------------------------------------------------------- H(x)      =  0.03786337215242741
 -------------------------------------------------------------- |grad(x)| =  0.33553119710096574

------[---- 3 ----]------

a =  0.00475325465423264
  x  =  [ 2.56655549  0.34994887 -0.09207751]
f(x) =  -21.529798166415354
 |grad(x)| =  0.07036203158326208

------[---- 4 ----]------

a =  0.005098984397592147
  x  =  [ 2.56639117  0.34970129 -0.09192403]
f(x) =  -21.529809932663834
 |grad(x)| =  0.05954515520991744

------[---- 5 ----]------

a =  0.004753276451734823
  x  =  [ 2.566469    0.34981687 -0.09165427]
f(x) =  -21.52981897223095
 |grad(x)| =  0.05405640141506334
сделано  5  шагов
 -------------------------------------------------------------- f(x) + ((  13.615773105863909  )) * H(x)    =     -21.527642453892035
 --------------------------------------------------------------   x       =  [ 2.56634287  0.34962659 -0.09153635]
 -------------------------------------------------------------- H(x)      =  0.016484150611506196
 -------------------------------------------------------------- |grad(x)| =  0.07687303080124787
-------------------------------------------------------

In [98]:
x0 = np.array([33/13, 4/13, 0])
e = 0.05
x_min = sanction(fx, gradfx, Hx, gradHx, x0, e)
print(round(fx(x_min), 3), x_min)

------------------------------------------------------------------------[---- 1 ----]------

------[---- 1 ----]------

a =  0.0043189248700947445
  x  =  [2.53846154 0.30769231 0.        ]
f(x) =  -21.307692307692307
 |grad(x)| =  8.875203139603666

------[---- 2 ----]------

a =  0.028317614877175597
  x  =  [2.55972394 0.33958591 0.        ]
f(x) =  -21.47779150257005
 |grad(x)| =  2.440276062264665

------[---- 3 ----]------

a =  0.004318924873538991
  x  =  [ 2.55972394  0.33958591 -0.0691028 ]
f(x) =  -21.562106454332817
 |grad(x)| =  4.399270223629821

------[---- 4 ----]------

a =  0.028317614672901215
  x  =  [ 2.57026331  0.35539496 -0.0691028 ]
f(x) =  -21.60389978012022
 |grad(x)| =  1.209598665603632

------[---- 5 ----]------

a =  0.004318924882826936
  x  =  [ 2.57026331  0.35539496 -0.10335575]
f(x) =  -21.624615940864615
 |grad(x)| =  2.1806349944825376

------[---- 6 ----]------

a =  0.028317616516843752
  x  =  [ 2.57548747  0.36323121 -0.10335575]
f(x) =  -21.63

a =  0.005082962363443561
  x  =  [ 2.56791005  0.35186507 -0.09651619]
f(x) =  -21.539189579320485
 |grad(x)| =  0.08858419234429646

------[---- 2 ----]------

a =  0.0051783876576023315
  x  =  [ 2.56768985  0.35153478 -0.09630366]
f(x) =  -21.53920952272813
 |grad(x)| =  0.07693835021202355

------[---- 3 ----]------

a =  0.005082958942176788
  x  =  [ 2.56779417  0.35169125 -0.09595242]
f(x) =  -21.539224849476422
 |grad(x)| =  0.06807806039172096

------[---- 4 ----]------

a =  0.005178382620849045
  x  =  [ 2.56762495  0.35143742 -0.09578909]
f(x) =  -21.53923662826682
 |grad(x)| =  0.05912804424219317

------[---- 5 ----]------

a =  0.005082959995672212
  x  =  [ 2.56770511  0.35155767 -0.09551916]
f(x) =  -21.53924568040842
 |grad(x)| =  0.05231875440179295
сделано  5  шагов
 -------------------------------------------------------------- f(x) + ((  13.071067811865476  )) * H(x)    =     -21.53669440426944
 --------------------------------------------------------------   x  

a =  0.0080744413422285
  x  =  [ 2.56122753  0.3418413  -0.07455806]
f(x) =  -21.487657548429034
 |grad(x)| =  0.05263068549632619

------[---- 2 ----]------

a =  0.0026422565957661993
  x  =  [ 2.56106411  0.34159616 -0.07425181]
f(x) =  -21.487668731511757
 |grad(x)| =  0.07634684400682117
сделано  2  шагов
 -------------------------------------------------------------- f(x) + ((  16.77032961426901  )) * H(x)    =     -21.48666281682779
 --------------------------------------------------------------   x       =  [ 2.56114475  0.34171712 -0.07411195]
 -------------------------------------------------------------- H(x)      =  0.010869710679906639
 -------------------------------------------------------------- |grad(x)| =  0.07893313278944405
------------------------------------------------------------------------[---- 55 ----]------

------[---- 1 ----]------

a =  0.0031145998131644472
  x  =  [ 2.56114475  0.34171712 -0.07411195]
f(x) =  -21.48666281682779
 |grad(x)| =  0.07893313